In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
test_FEN = "rnbqkbnr/1ppppppp/p7/7P/8/8/PPPPPPP1/RNBQKBNR b KQkq - 0 2"

def extract_elements(string):
    elements = string.split(' ')
    board = elements[0]
    rest = elements[1:]
    return board, rest

def board_by_rank(board):
    return board.split('/')

ind_to_file = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h'}
ind_to_rank = {0:'8', 1:'7', 2:'6', 3:'5', 4:'4', 5:'3', 6:'2', 7:'1'}

board, rest = extract_elements(test_FEN)
board = board.replace("1", "X")
board = board.replace("2", "XX")
board = board.replace("3", "XXX")
board = board.replace("4", "XXXX")
board = board.replace("5", "XXXXX")
board = board.replace("6", "XXXXXX")
board = board.replace("7", "XXXXXXX")
board = board.replace("8", "XXXXXXXX")
board = board_by_rank(board)

def has_moved(piece, color, pos):
    if (piece == 'K') and (color == 'White'):
        starts = ['e1']
    elif (piece == 'K') and (color == 'Black'):
        starts = ['e8']
    if (piece == 'Q') and (color == 'White'):
        starts = ['d1']
    elif (piece == 'Q') and (color == 'Black'):
        starts = ['d8']
    if (piece == 'R') and (color == 'White'):
        starts = ['a1', 'h1']
    elif (piece == 'R') and (color == 'Black'):
        starts = ['a8', 'h8']
    if (piece == 'N') and (color == 'White'):
        starts = ['b1', 'g1']
    elif (piece == 'N') and (color == 'Black'):
        starts = ['b8', 'g8']
    if (piece == 'B') and (color == 'White'):
        starts = ['c1', 'f1']
    elif (piece == 'B') and (color == 'Black'):
        starts = ['c8', 'f8']
    if (piece == 'P') and (color == 'White'):
        starts = ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2']
    elif (piece == 'P') and (color == 'Black'):
        starts = ['a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7']
        
    return not pos in starts
    
pieces = []
colors = []
squares = []
moved = []
for i in range(8):
    for j in range(8):
        char = board[i][j]
        if (char != "X"):
            id_str = str.upper(char)
            if (id_str == char):
                color = 'White'
            else:
                color = 'Black'
            position = ind_to_file[j] + ind_to_rank[i]
            has_move = has_moved(id_str, color, position)
            pieces.append(id_str)
            colors.append(color)
            squares.append(position)
            moved.append(has_move)
        
piece_df = pd.DataFrame({'piece': pieces, 'color': colors, 'pos':squares, 'moved':moved})
grouped_piece_df = piece_df.groupby(['piece', 'color']).agg({'pos': list, 'moved':list}).reset_index()

In [3]:
grouped_piece_df;

In [4]:
active_pieces = []
for index, row in grouped_piece_df.iterrows():
    active_pieces.append({'id': row['piece'], 'color': row['color'], 'positions': row['pos'], 'has_moved': row['moved']})

In [5]:
active_pieces;

In [6]:
full_json = {'active_pieces': active_pieces, 'captured_pieces' : []}
full_json;

In [7]:
json.dumps(full_json);

In [7]:
with open('test_board_jsons/pawn_cant_pin.json', 'w') as f:
    json.dump(full_json, f)